# P2SH MultiSig BIP322
## Role: Creator / Updator

In [1]:
cd ..

/home/will/work/LegendaryRequirements/clients/dcd/bip0322-sigs


## 1. Create HD Key from mnemonic

Note: this would typically access some wallet

In [2]:
from buidl.mnemonic import secure_mnemonic
from buidl.hd import HDPrivateKey


In [3]:
mnemonic = 'priority orange artist purpose sport blade spike leg calm renew dress figure session play solve fatal spy reform humble carry finger response order print'

In [4]:
hd_rootkey = HDPrivateKey.from_mnemonic(mnemonic)

root_fp = hd_rootkey.fingerprint().hex()

## 2. Select Key to Sign With

In [5]:
bip32_path = "m/45'/0/0/1"

hd_priv_a = hd_rootkey.traverse(bip32_path)
pubkey = hd_priv_a.pub.point


## 3. Fetch Other Signing Parties Public Keys

See [signer_1](./Signer_1.ipynb) and [signer_2](./Signer_2.ipynb) notebooks.

In [6]:
signer_1_pubkey = '02fd5142d699b3bfaf2aa41e5a5dab6ff8ddfe03319048d49e365d0c63c366430b'
signer_2_pubkey = '02fae5c1f65f93bb2ee8e34cb0cbda5f0fc72d18d94511f7dbb5a827e3773edd92'


## 4. Create 2-of-3 MultiSig Pay-to-Script Hash

Note: This p2sh_address would likely be something that the signers would want to include in a DID Document.

In [7]:
from buidl.script import RedeemScript

# YOU COULD CHANGE THIS
quorum_m=2

redeem_script = RedeemScript.create_p2sh_multisig(
            quorum_m=quorum_m,
            pubkey_hexes=[pubkey.sec().hex(),signer_1_pubkey,signer_2_pubkey],
            sort_keys=False,
        )


script_pubkey = redeem_script.script_pubkey()
p2sh_address = redeem_script.address()

In [8]:
print("The BIP322 to_spend tx will be `locked` with a script_pubkey with the following address : ", p2sh_address)

The BIP322 to_spend tx will be `locked` with a script_pubkey with the following address :  3LnYoUkFrhyYP3V7rq3mhpwALz1XbCY9Uq


## 5. Create the virtual to_spend transaction locked with the p2sh_multisig

BIP 0322 defines the structure of this transaction as follows:

```python
    nVersion = 0
    nLockTime = 0
    vin[0].prevout.hash = 0000...000
    vin[0].prevout.n = 0xFFFFFFFF
    vin[0].nSequence = 0
    vin[0].scriptSig = OP_0 PUSH32[ message_hash ]
    vin[0].scriptWitness = []
    vout[0].nValue = 0
    vout[0].scriptPubKey = message_challenge
```

The message_challenge is the above 2-3 p2sh scriptPubKey

In [9]:
from src.message import create_to_spend_tx

## You could change this
message_to_sign = "This will be a p2sh 2-of-3 multisig BIP 322 signed message"

to_spend_tx = create_to_spend_tx(p2sh_address, message_to_sign)

## 6. Create PSBT to_sign transaction

In [10]:
from src.message import create_to_sign_tx

to_sign_tx = create_to_sign_tx(to_spend_tx.hash())


In [11]:
pubkey_records = []
input_dicts = []
output_dicts = []

In [37]:
from buidl.psbt_helper import create_multisig_psbt
from buidl.psbt import PSBT

fee = 0
# psbt_to_sign_bip322 = create_multisig_psbt(pubkey_records,input_dicts,output_dicts,fee)
psbt_to_sign_bip322 = PSBT.create(to_sign_tx)

In [44]:
psbt_to_sign_bip322


Tx:

tx: 9723b96cd3d5b4af6c93ca302268ff92bf35933fbaee8662a2b465b854ebba99
version: 0
locktime: 0
tx_ins:
64b5fb9b8d067a78cf282396a6a042fb85c6af13680b99ff7f7cf935daf171cd:0
tx_outs:
0:OP_RETURN 


PSBT XPUBS:
{}

Psbt_Ins:
[
TxIn:
64b5fb9b8d067a78cf282396a6a042fb85c6af13680b99ff7f7cf935daf171cd:0
Prev Tx:
None
Prev Output:
None
Sigs:
{}
RedeemScript:
None
WitnessScript:
None
PSBT Pubs:
{}
ScriptSig:
None
Witness:
None
]

Psbt_Outs:
[
TxOut:
0:OP_RETURN 
RedeemScript:
None
WitnessScript
None
PSBT Pubs:
{}
]

Extra:
{}

In [50]:
from io import BytesIO

b64_psbt = psbt_to_sign_bip322.serialize_base64()
b64_psbt

# stream = BytesIO(b64_psbt)
PSBT.parse_base64(b64_psbt)

mismatch between length and consumed bytes 55 vs 52


OSError: stream has no bytes

In [40]:
len(b64_psbt)

100

In [43]:
test = 'cHNidP8BAD8CAAAAAf//////////////////////////////////////////AAAAAAD/////AQAAAAAAAAAAA2oBAAAAAAAACg8BAgMEBQYHCAkPAQIDBAUGBwgJCgsMDQ4PAAA='
PSBT.parse_base64(test)
# len(test)


Tx:

tx: 75c5c9665a570569ad77dd1279e6fd4628a093c4dcbf8d41532614044c14c115
version: 2
locktime: 0
tx_ins:
ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff:0
tx_outs:
0:OP_RETURN 00 


PSBT XPUBS:
{}

Psbt_Ins:
[
TxIn:
ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff:0
Prev Tx:
None
Prev Output:
None
Sigs:
{}
RedeemScript:
None
WitnessScript:
None
PSBT Pubs:
{}
ScriptSig:
None
Witness:
None
]

Psbt_Outs:
[
TxOut:
0:OP_RETURN 00 
RedeemScript:
None
WitnessScript
None
PSBT Pubs:
{}
]

Extra:
{}

In [17]:
tx_lookup = {}

tx_lookup[to_spend_tx.hash()] = to_spend_tx

In [18]:
from buidl.psbt import NamedHDPublicKey

pubkey_lookup = {}

named_hd_pubkey_obj = NamedHDPublicKey.from_hd_pub(
    child_hd_pub=hd_priv_a.pub,
    xfp_hex=root_fp,
    path=bip32_path,
)
# pubkey lookups needed for validation
pubkey_lookup[named_hd_pubkey_obj.sec()] = named_hd_pubkey_obj

In [25]:
redeem_lookup = {}
redeem_lookup[redeem_script.hash160()] = redeem_script

In [26]:
psbt_to_sign_bip322.update(tx_lookup,pubkey_lookup,redeem_lookup)

In [27]:
psbt_to_sign_bip322


Tx:

tx: 9723b96cd3d5b4af6c93ca302268ff92bf35933fbaee8662a2b465b854ebba99
version: 0
locktime: 0
tx_ins:
64b5fb9b8d067a78cf282396a6a042fb85c6af13680b99ff7f7cf935daf171cd:0
tx_outs:
0:OP_RETURN 


PSBT XPUBS:
{}

Psbt_Ins:
[
TxIn:
64b5fb9b8d067a78cf282396a6a042fb85c6af13680b99ff7f7cf935daf171cd:0
Prev Tx:

tx: 64b5fb9b8d067a78cf282396a6a042fb85c6af13680b99ff7f7cf935daf171cd
version: 0
locktime: 0
tx_ins:
0000000000000000000000000000000000000000000000000000000000000000:4294967295
tx_outs:
0:OP_HASH160 d1763eda828dbe7ae8374d2d7fb6c3381970b272 OP_EQUAL 

Prev Output:
None
Sigs:
{}
RedeemScript:
OP_2 022897c6cb6f2298b72e01e3412c708dbf73513f98e72701433515c260d63fbc9e 02fd5142d699b3bfaf2aa41e5a5dab6ff8ddfe03319048d49e365d0c63c366430b 02fae5c1f65f93bb2ee8e34cb0cbda5f0fc72d18d94511f7dbb5a827e3773edd92 OP_3 OP_CHECKMULTISIG 
WitnessScript:
None
PSBT Pubs:
{b'\x02(\x97\xc6\xcbo"\x98\xb7.\x01\xe3A,p\x8d\xbfsQ?\x98\xe7\'\x01C5\x15\xc2`\xd6?\xbc\x9e': 
Point: 022897c6cb6f2298b72e01e3412c708dbf73513

In [35]:
from io import BytesIO

b64_psbt = psbt_to_sign_bip322.serialize_base64()
b64_psbt

'cHNidP8BAEAAAAAAAAEBzXHx2jX5fH//mQtoE6/GhftCoKaWIyjPeHoGjZv7tWQAAAAAAAAAAAABAAAAAAAAAAABagAAAAAAAAEAeAAAAAAAAQEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAP////8iACA4E3j7MVlbnxgcxmUY+N1w459TuGihtX+nhDqV8uyuxQAAAAABAAAAAAAAAAAXqRTRdj7ago2+eug3TS1/tsM4GXCycocAAAAAAAEEaVIhAiiXxstvIpi3LgHjQSxwjb9zUT+Y5ycBQzUVwmDWP7yeIQL9UULWmbO/ryqkHlpdq2/43f4DMZBI1J42XQxjw2ZDCyEC+uXB9l+Tuy7o40ywy9pfD8ctGNlFEffbtagn43c+3ZJTriIGAiiXxstvIpi3LgHjQSxwjb9zUT+Y5ycBQzUVwmDWP7yeFHsF6hstAACAAAAAAAAAAAABAAAAAAA='

In [36]:
PSBT.parse_base64(b64_psbt)

mismatch between length and consumed bytes 131 vs 127



Tx:

tx: 9beadacd9baf9f68d6084be6862f42ae01953e2d6f9479b87fdae23538595f37
version: 0
locktime: 3316575474
tx_ins:

tx_outs:
9005288144373927169:OP_[255] OP_[153] 6813afc685fb42a0a69623 cf787a068d9bfbb564000000000000000000010000000000000000016a0000000000000100780000 OP_0 OP_0 OP_0 01 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_[255] OP_[255] OP_[255] OP_[255] 0020381378fb31595b9f181cc66518f8dd70e39f53b868a1b57fa7843a95 


PSBT XPUBS:
{}

Psbt_Ins:
[]

Psbt_Outs:
[
TxOut:
9005288144373927169:OP_[255] OP_[153] 6813afc685fb42a0a69623 cf787a068d9bfbb564000000000000000000010000000000000000016a0000000000000100780000 OP_0 OP_0 OP_0 01 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_0 OP_[255] OP_[255] OP_[255] OP_[255] 0020381378fb31595b9f181cc66518f8dd70e39f53b868a1b

In [33]:
b64_psbt

b'psbt\xff\x01\x00@\x00\x00\x00\x00\x00\x01\x01\xcdq\xf1\xda5\xf9|\x7f\xff\x99\x0bh\x13\xaf\xc6\x85\xfbB\xa0\xa6\x96#(\xcfxz\x06\x8d\x9b\xfb\xb5d\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01j\x00\x00\x00\x00\x00\x00\x01\x00x\x00\x00\x00\x00\x00\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\xff"\x00 8\x13x\xfb1Y[\x9f\x18\x1c\xc6e\x18\xf8\xddp\xe3\x9fS\xb8h\xa1\xb5\x7f\xa7\x84:\x95\xf2\xec\xae\xc5\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x17\xa9\x14\xd1v>\xda\x82\x8d\xbez\xe87M-\x7f\xb6\xc38\x19p\xb2r\x87\x00\x00\x00\x00\x00\x01\x04iR!\x02(\x97\xc6\xcbo"\x98\xb7.\x01\xe3A,p\x8d\xbfsQ?\x98\xe7\'\x01C5\x15\xc2`\xd6?\xbc\x9e!\x02\xfdQB\xd6\x99\xb3\xbf\xaf*\xa4\x1eZ]\xabo\xf8\xdd\xfe\x031\x90H\xd4\x9e6]\x0cc\xc3fC\x0b!\x02\xfa\xe5\xc1\xf6_\x93\xbb.\xe8\xe3L\xb0\xcb\xda_\x0f\xc7-\x18\xd9E\x11\xf7\xdb\xb5\xa8\'\xe3w>\xdd\x92S\xae"\x06\x02(\x97\xc6\xcbo

In [ ]:
psbt